In [2]:
%pip install pandas numpy matplotlib seaborn folium haversine scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster, HeatMap
from sklearn.cluster import DBSCAN
from datetime import datetime
import matplotlib.font_manager as fm
import os
from haversine import haversine
import warnings
warnings.filterwarnings('ignore')

In [4]:
# 한글 폰트 설정 - Windows 환경
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 데이터 로드
try:
    reports = pd.read_csv('reports.csv', encoding='utf-8')
    parks = pd.read_csv('parks.csv', encoding='utf-8')
    print("데이터 로드 성공!")
except Exception as e:
    print(f"데이터 로드 중 오류 발생: {e}")
    # CP949도 시도
    try:
        reports = pd.read_csv('reports.csv', encoding='cp949')
        parks = pd.read_csv('parks.csv', encoding='cp949')
        print("CP949 인코딩으로 데이터 로드 성공!")
    except Exception as e2:
        print(f"CP949 인코딩으로도 실패: {e2}")

데이터 로드 성공!


In [5]:
print("\n----- 민원 데이터 기본 정보 -----")
print(f"데이터 크기: {reports.shape}")
print(reports.info())
print(reports.head())


----- 민원 데이터 기본 정보 -----
데이터 크기: (3064538, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3064538 entries, 0 to 3064537
Data columns (total 6 columns):
 #   Column  Dtype  
---  ------  -----  
 0   민원접수일   object 
 1   민원접수시간  object 
 2   주소      object 
 3   경도      float64
 4   위도      float64
 5   요일      object 
dtypes: float64(2), object(4)
memory usage: 140.3+ MB
None
        민원접수일    민원접수시간                       주소          경도         위도  \
0  2021-09-29  19:29:00      서울특별시 강서구 강서로15길 49  126.843247  37.532089   
1  2021-09-29  18:48:00        성북구 오패산로19길 34-5   127.033761  37.609537   
2  2021-09-29  18:47:00  장위로21다길 59-19 주소지 앞도로 외  127.045741  37.616406   
3  2021-09-29  18:47:00     서울특별시 강북구 오패산로30길 13  127.034685  37.613820   
4  2021-09-29  18:46:00    서울특별시 강서구 강서로18길 52-5  126.848703  37.534293   

        요일  
0  Weekday  
1  Weekday  
2  Weekday  
3  Weekday  
4  Weekday  


In [7]:
print("\n----- 주차장 데이터 기본 정보 -----")
print(f"데이터 크기: {parks.shape}")
print(parks.info())
print(parks.head())


----- 주차장 데이터 기본 정보 -----
데이터 크기: (1463, 20)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1463 entries, 0 to 1462
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   주소      1463 non-null   object 
 1   주차장종류   1463 non-null   object 
 2   운영구분명   1463 non-null   object 
 3   총주차면    1463 non-null   float64
 4   평일유료    1463 non-null   object 
 5   토요일유료   1463 non-null   object 
 6   공휴일유료   1463 non-null   object 
 7   평일시작    1463 non-null   object 
 8   평일종료    1463 non-null   object 
 9   토요일시작   1463 non-null   object 
 10  토요일종료   1463 non-null   object 
 11  공휴일시작   1463 non-null   object 
 12  공휴일종료   1463 non-null   object 
 13  기본주차요금  1463 non-null   float64
 14  기본주차시간  1463 non-null   float64
 15  추가단위요금  1463 non-null   float64
 16  추가단위시간  1463 non-null   float64
 17  경도      1463 non-null   float64
 18  위도      1463 non-null   float64
 19  1시간 요금  1463 non-null   float64
dtypes: float64(8), object(12)
me

In [8]:
def preprocess_data():
    # 민원 데이터 전처리
    # 날짜, 시간 형식 변환
    reports['민원접수일시'] = pd.to_datetime(reports['민원접수일'] + ' ' + reports['민원접수시간'])
    reports['월'] = reports['민원접수일시'].dt.month
    reports['일'] = reports['민원접수일시'].dt.day
    reports['시간'] = reports['민원접수일시'].dt.hour
    
    # 주차장 데이터 전처리
    # 요금 관련 컬럼 숫자로 변환
    for col in ['기본주차요금', '기본주차시간', '추가단위요금', '추가단위시간', '1시간 요금', '총주차면']:
        if col in parks.columns:
            parks[col] = pd.to_numeric(parks[col], errors='coerce')
    
    # 운영 시간 전처리
    time_cols = ['평일시작', '평일종료', '토요일시작', '토요일종료', '공휴일시작', '공휴일종료']
    for col in time_cols:
        if col in parks.columns:
            # 시간 형식 통일 (예: '0900' -> '09:00')
            parks[col] = parks[col].astype(str)
            parks[col] = parks[col].apply(lambda x: 
                                      x.zfill(4) if x.isdigit() and len(x) <= 4 
                                      else x)
            parks[col] = parks[col].apply(lambda x: 
                                      f"{x[:2]}:{x[2:]}" if x.isdigit() and len(x) == 4 
                                      else x)
    
    # 유료 여부 이진화
    binary_cols = ['평일유료', '토요일유료', '공휴일유료']
    for col in binary_cols:
        if col in parks.columns:
            parks[col] = parks[col].map({'Y': 1, 'N': 0})
    
    return reports, parks

reports, parks = preprocess_data()
print("데이터 전처리 완료!")

데이터 전처리 완료!


In [9]:
# 기본 통계적 분석
def basic_statistics():
    print("\n----- 기본 통계 분석 -----")
    
    # 민원 데이터 통계
    print("\n민원 데이터 통계:")
    print(f"총 민원 수: {len(reports)}")
    
    reports_per_day = reports.groupby('민원접수일').size()
    print(f"일평균 민원 수: {reports_per_day.mean():.2f}")
    print(f"최대 민원 발생일: {reports_per_day.idxmax()}, 건수: {reports_per_day.max()}")
    
    reports_per_weekday = reports.groupby('요일').size()
    print("\n요일별 민원 발생 건수:")
    print(reports_per_weekday)
    
    # 주차장 데이터 통계
    print("\n주차장 데이터 통계:")
    print(f"총 주차장 수: {len(parks)}")
    
    if '주차장종류' in parks.columns:
        type_counts = parks['주차장종류'].value_counts()
        print("\n주차장 종류별 수:")
        print(type_counts)
    
    if '총주차면' in parks.columns:
        print(f"\n총 주차면 수: {parks['총주차면'].sum()}")
        print(f"주차장당 평균 주차면 수: {parks['총주차면'].mean():.2f}")
    
    if '1시간 요금' in parks.columns:
        print(f"\n평균 1시간 주차 요금: {parks['1시간 요금'].mean():.2f}원")
        print(f"최소 1시간 주차 요금: {parks['1시간 요금'].min()}원")
        print(f"최대 1시간 주차 요금: {parks['1시간 요금'].max()}원")

basic_statistics()


----- 기본 통계 분석 -----

민원 데이터 통계:
총 민원 수: 3064538
일평균 민원 수: 3427.89
최대 민원 발생일: 2023-04-23, 건수: 9669

요일별 민원 발생 건수:
요일
Holiday      546988
Saturday     490527
Weekday     2027023
dtype: int64

주차장 데이터 통계:
총 주차장 수: 1463

주차장 종류별 수:
주차장종류
NS    757
NW    706
Name: count, dtype: int64

총 주차면 수: 82780.0
주차장당 평균 주차면 수: 56.58

평균 1시간 주차 요금: 1985.55원
최소 1시간 주차 요금: 0.0원
최대 1시간 주차 요금: 9200.0원


In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

def temporal_analysis(reports: pd.DataFrame, output_path: str = 'temporal_analysis.png'):
    # 1) 복사본 생성
    df = reports.copy()

    # 2) 날짜+시간 → datetime
    df['접수일시'] = pd.to_datetime(df['민원접수일'].astype(str) + ' ' + df['시간'].astype(str))

    # 3) 요일, 시간대(hour), 월(month) 파생
    dow_map = {0:'월요일', 1:'화요일', 2:'수요일', 3:'목요일',
               4:'금요일', 5:'토요일', 6:'일요일'}
    df['요일']   = df['접수일시'].dt.dayofweek.map(dow_map)
    df['시간대'] = df['접수일시'].dt.hour
    df['월']     = df['접수일시'].dt.month

    # 4) 플롯 세팅
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    sns.set_style('whitegrid')

    # 4-1) 요일별
    ax = axes[0,0]
    order = ['월요일','화요일','수요일','목요일','금요일','토요일','일요일']
    sns.countplot(x='요일', data=df, order=order, ax=ax)
    ax.set_title('요일별 민원 발생 건수')
    ax.set_xlabel('')
    ax.set_ylabel('건수')
    ax.tick_params(axis='x', rotation=45)

    # 4-2) 시간대별
    ax = axes[0,1]
    sns.countplot(x='시간대', data=df, order=list(range(24)), ax=ax)
    ax.set_title('시간대별 민원 발생 건수')
    ax.set_xlabel('시간 (시)')
    ax.set_ylabel('건수')

    # 4-3) 월별
    ax = axes[1,0]
    sns.countplot(x='월', data=df, order=list(range(1,13)), ax=ax)
    ax.set_title('월별 민원 발생 건수')
    ax.set_xlabel('월')
    ax.set_ylabel('건수')

    # 4-4) 일별 추이
    ax = axes[1,1]
    daily = df.set_index('접수일시').resample('D').size()
    ax.plot(daily.index, daily.values, marker='o', linestyle='-')
    ax.set_title('일별 민원 발생 추이')
    ax.set_xlabel('날짜')
    ax.set_ylabel('건수')
    ax.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

    plt.tight_layout()
    plt.savefig(output_path, dpi=300)
    plt.close(fig)
    print(f"시간적 분석 완료! '{output_path}'로 저장되었습니다.")

temporal_analysis(reports)

시간적 분석 완료! 'temporal_analysis.png'로 저장되었습니다.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def proximity_analysis(parks, reports, sample_size=1000):
    # 1) 유효 좌표만 필터링
    valid_reports = reports.dropna(subset=['위도','경도']).reset_index(drop=True)
    valid_parks   = parks.dropna(subset=['위도','경도']).reset_index(drop=True)
    
    # 2) 민원 샘플링 (속도 개선용)
    if len(valid_reports) > sample_size:
        sampled = valid_reports.sample(sample_size, random_state=42).reset_index(drop=True)
    else:
        sampled = valid_reports.copy()
    
    # 3) 좌표 배열로 변환
    rep_coords  = sampled[['위도','경도']].to_numpy()  # shape (N_reports, 2)
    park_coords = valid_parks[['위도','경도']].to_numpy()  # shape (N_parks,   2)
    
    # 4) 벡터화된 haversine 거리 계산 함수
    def haversine_array(lat, lon, coords):
        R = 6_371_000  # 지구 반지름 (m)
        φ1 = np.radians(lat)
        λ1 = np.radians(lon)
        φ2 = np.radians(coords[:,0])
        λ2 = np.radians(coords[:,1])
        dφ = φ2 - φ1
        dλ = λ2 - λ1
        a = np.sin(dφ/2)**2 + np.cos(φ1)*np.cos(φ2)*np.sin(dλ/2)**2
        return 2 * R * np.arcsin(np.sqrt(a))  # shape (coords.shape[0],)
    
    # 5) 각 민원마다 가장 가까운 주차장 거리 및 주소 구하기
    distances     = []
    nearest_park  = []
    for lat, lon in rep_coords:
        dists = haversine_array(lat, lon, park_coords)
        idx   = np.argmin(dists)  # 최소 거리 인덱스
        distances.append(dists[idx])
        nearest_park.append(valid_parks.loc[idx, '주소'])
    
    distances = np.array(distances)
    
    # 6) 히스토그램: 거리 분포
    plt.figure(figsize=(10,6))
    plt.hist(distances, bins=30, alpha=0.7)
    plt.axvline(distances.mean(), color='r', linestyle='--',
                label=f'평균: {distances.mean():.1f} m')
    plt.axvline(np.median(distances), color='g', linestyle='--',
                label=f'중앙값: {np.median(distances):.1f} m')
    plt.title('민원 위치 ↔ 최단 주차장 거리 분포')
    plt.xlabel('거리 (m)')
    plt.ylabel('샘플 민원 수')
    plt.legend()
    plt.tight_layout()
    plt.savefig('proximity_analysis.png', dpi=300)
    plt.close()
    
    # 7) 거리 구간별 민원 분포
    bins   = [0, 100, 200, 500, 1000, np.inf]
    labels = ['0-100m','100-200m','200-500m','500-1000m','1000m+']
    cat    = pd.cut(distances, bins=bins, labels=labels)
    counts = cat.value_counts().sort_index()
    
    plt.figure(figsize=(10,6))
    sns.barplot(x=counts.index, y=counts.values)
    plt.title('주차장과의 거리별 민원 분포')
    plt.xlabel('거리 구간')
    plt.ylabel('샘플 민원 수')
    plt.tight_layout()
    plt.savefig('distance_distribution.png', dpi=300)
    plt.close()
    
    # 8) 간단한 통계 출력
    print("근접성 분석 완료!")
    print(f" - 샘플 크기: {len(distances)} 건")
    print(f" - 평균 거리: {distances.mean():.1f} m")
    print(f" - 중앙값 거리: {np.median(distances):.1f} m")
    print("생성된 파일: proximity_analysis.png, distance_distribution.png")

# 사용 예시
proximity_analysis(parks, reports)


In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial import cKDTree

def proximity_analysis(
    parks: pd.DataFrame,
    reports: pd.DataFrame,
    sample_size: int = 1000,
    save_plots: bool = True,
    inline_plot: bool = False
) -> pd.DataFrame:
    """
    - parks, reports 에서 '위도','경도' 유효치 필터링
    - reports 를 sample_size만큼 샘플링
    - cKDTree 로 최근접 주차장 찾기 (거리 m 단위)
    - 거리분포 히스토그램, 구간별 막대차트 생성
    - (옵션) 이미지 저장 / 인라인 표시
    - 최종적으로 각 민원별 거리·추가정보 DataFrame 반환
    """
    # 1) 좌표 유효치 필터링
    rpt = reports.dropna(subset=['위도','경도']).reset_index(drop=True)
    prk = parks.dropna(subset=['위도','경도']).reset_index(drop=True)
    if rpt.empty or prk.empty:
        raise ValueError("유효한 민원 혹은 주차장 좌표가 없습니다.")
    
    # 2) 샘플링
    if len(rpt) > sample_size:
        rpt_s = rpt.sample(sample_size, random_state=42).reset_index(drop=True)
    else:
        rpt_s = rpt.copy()
    
    # 3) KD-Tree 생성
    park_coords = np.radians(prk[['위도','경도']].to_numpy())
    tree = cKDTree(park_coords)
    
    # 4) 민원 좌표 배열 (radians)
    rep_coords = np.radians(rpt_s[['위도','경도']].to_numpy())
    
    # 5) 최근접 탐색 (지구 반지름 곱해서 거리 m로 변환)
    R = 6_371_000
    dists_rad, idxs = tree.query(rep_coords)
    distances = dists_rad * R
    nearest_addr = prk.loc[idxs, '주소'].values
    
    # 6) 결과 DataFrame
    result_df = rpt_s.copy()
    result_df['nearest_addr'] = nearest_addr
    result_df['distance_m']    = distances
    
    # 7) 플롯 함수
    def _plot():
        # 히스토그램
        fig, ax = plt.subplots(figsize=(8,4))
        ax.hist(distances, bins=30, alpha=0.7)
        ax.axvline(distances.mean(), color='r', linestyle='--',
                   label=f'평균 {distances.mean():.0f} m')
        ax.axvline(np.median(distances), color='g', linestyle='--',
                   label=f'중앙값 {np.median(distances):.0f} m')
        ax.set_title('민원→주차장 최단거리 분포')
        ax.set_xlabel('거리 (m)')
        ax.set_ylabel('민원 수')
        ax.legend()
        plt.tight_layout()
        if save_plots: plt.savefig('proximity_hist.png', dpi=300)
        if inline_plot: plt.show()
        plt.close(fig)
        
        # 구간별 막대차트
        bins   = [0,100,200,500,1000,np.inf]
        labels = ['0-100m','100-200m','200-500m','500-1000m','1000m+']
        cat    = pd.cut(distances, bins=bins, labels=labels)
        counts = cat.value_counts().sort_index()
        
        fig, ax = plt.subplots(figsize=(8,4))
        sns.barplot(x=counts.index, y=counts.values, ax=ax)
        ax.set_title('거리 구간별 민원 분포')
        ax.set_xlabel('거리 구간')
        ax.set_ylabel('민원 수')
        plt.tight_layout()
        if save_plots: plt.savefig('proximity_bins.png', dpi=300)
        if inline_plot: plt.show()
        plt.close(fig)
    
    _plot()
    
    # 8) 통계 출력
    print("근접성 분석 완료!")
    print(f" - 샘플링 민원 수: {len(distances)}")
    print(f" - 평균 거리: {distances.mean():.1f} m")
    print(f" - 중앙값 거리: {np.median(distances):.1f} m")
    
    return result_df

df = proximity_analysis(parks, reports, sample_size=2000, save_plots=False, inline_plot=False)
df.head()


근접성 분석 완료!
 - 샘플링 민원 수: 2000
 - 평균 거리: 258812906.3 m
 - 중앙값 거리: 313.2 m


,민원접수일,민원접수시간,주소,경도,위도,요일,민원접수일시,월,일,시간,nearest_addr,distance_m
0,2022-09-14,15:21:00,서울 송파구 장지동 917-6,127.143405,37.474078,Weekday,2022-09-14 15:21:00,9,14,15,송파구 장지동 600-2,1799.162127
1,2023-01-27,14:23:00,상도로 40,126.924988,37.499601,Weekday,2023-01-27 14:23:00,1,27,14,동작구 대방동 395-1,157.624400
2,2023-06-18,12:55:00,서울특별시 강동구 동남로71길 32,127.154672,37.552114,Holiday,2023-06-18 12:55:00,6,18,12,강동구 명일동 51-1,45.654170
3,2023-01-28,05:26:00,서초구 서운로 178 서초초등학교 앞,127.024892,37.499049,Saturday,2023-01-28 05:26:00,1,28,5,강남구 역삼동 635-1,577.662084
4,2022-05-18,20:18:00,서울특별시 종로구 성균관로14길 60,126.997668,37.586799,Weekday,2022-05-18 20:18:00,5,18,20,종로구 명륜3가 53-0,100.990326


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def parking_characteristics_analysis(parks, reports):
    # 1) 유효 좌표만 필터
    valid_reports = reports.dropna(subset=['위도','경도'])
    valid_parks   = parks.dropna(subset=['위도','경도']).reset_index(drop=True)

    # 2) 벡터 거리 계산용 보고서 배열 한 번만
    rep_coords = valid_reports[['위도','경도']].to_numpy()

    # 3) haversine 함수(벡터화)
    def haversine_array(lat, lon, coords):
        R = 6371000  # m
        φ1 = np.radians(lat)
        λ1 = np.radians(lon)
        φ2 = np.radians(coords[:,0])
        λ2 = np.radians(coords[:,1])
        dφ = φ2 - φ1
        dλ = λ2 - λ1
        a = np.sin(dφ/2)**2 + np.cos(φ1)*np.cos(φ2)*np.sin(dλ/2)**2
        return 2 * R * np.arcsin(np.sqrt(a))

    radius = 500  # m
    complaint_counts = []

    # 4) 각 주차장별 민원 수 계산
    for _, park in valid_parks.iterrows():
        lat, lon = park['위도'], park['경도']
        dists = haversine_array(lat, lon, rep_coords)
        cnt   = int((dists <= radius).sum())
        complaint_counts.append(cnt)

    # 5) 결과 DataFrame에 합치기
    df = valid_parks.copy()
    df['민원 수'] = complaint_counts

    # 6) 주차장 종류별 평균 민원 수
    if '주차장종류' in df:
        plt.figure(figsize=(12,6))
        order = df.groupby('주차장종류')['민원 수'].mean().sort_values(ascending=False).index
        sns.barplot(data=df, x='주차장종류', y='민원 수', order=order)
        plt.title(f'주차장 종류별 {radius}m 반경 내 평균 민원 수')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig('parking_type_complaints.png')
        plt.close()

    # 7) 요금대별 평균 민원 수
    if '1시간 요금' in df:
        df_price = df.dropna(subset=['1시간 요금']).copy()
        bins  = [-0.1,0,500,1000,2000,3000,5000,np.inf]
        labels= ['무료','~500','~1000','~2000','~3000','~5000','5000+']
        df_price['요금대'] = pd.cut(df_price['1시간 요금'], bins=bins, labels=labels)
        plt.figure(figsize=(10,6))
        order = df_price.groupby('요금대')['민원 수'].mean().sort_values(ascending=False).index
        sns.barplot(data=df_price, x='요금대', y='민원 수', order=order)
        plt.title(f'요금대별 {radius}m 반경 내 평균 민원 수')
        plt.tight_layout()
        plt.savefig('parking_price_complaints.png')
        plt.close()

    # 8) 규모별 평균 민원 수
    if '총주차면' in df:
        df_size = df.dropna(subset=['총주차면']).copy()
        bins  = [0,10,30,50,100,200,np.inf]
        labels= ['~10면','~30면','~50면','~100면','~200면','200면+']
        df_size['규모대'] = pd.cut(df_size['총주차면'], bins=bins, labels=labels)
        plt.figure(figsize=(10,6))
        order = df_size.groupby('규모대')['민원 수'].mean().sort_values(ascending=False).index
        sns.barplot(data=df_size, x='규모대', y='민원 수', order=order)
        plt.title(f'주차장 규모별 {radius}m 반경 내 평균 민원 수')
        plt.tight_layout()
        plt.savefig('parking_size_complaints.png')
        plt.close()

    print("분석 완료! 아래 세 파일이 생성되었습니다:")
    print(" - parking_type_complaints.png\n - parking_price_complaints.png\n - parking_size_complaints.png")

# 사용 예
parking_characteristics_analysis(parks, reports)


분석 완료! 아래 세 파일이 생성되었습니다:
 - parking_type_complaints.png
 - parking_price_complaints.png
 - parking_size_complaints.png


In [21]:
# 9. 종합 분석 및 추천
def comprehensive_analysis():
    print("\n====== 종합 분석 및 추천 ======")
    
    # 요일별, 시간대별 민원 분석을 통한 주차 수요 예측
    if '요일' in reports.columns and '시간' in reports.columns:
        # 요일/시간대별 민원 발생 히트맵
        pivot_table = pd.pivot_table(
            reports, 
            values='민원접수일시', 
            index='요일', 
            columns='시간', 
            aggfunc='count',
            fill_value=0
        )
        
        # 요일 순서 정렬
        weekday_order = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
        if all(day in pivot_table.index for day in weekday_order):
            pivot_table = pivot_table.reindex(weekday_order)
        
        plt.figure(figsize=(12, 8))
        sns.heatmap(pivot_table, cmap='YlOrRd', annot=True, fmt='g')
        plt.title('요일 및 시간대별 민원 발생 히트맵')
        plt.xlabel('시간')
        plt.ylabel('요일')
        
        plt.tight_layout()
        plt.savefig('complaint_time_heatmap.png', dpi=300)
        plt.close()
        
        print("요일 및 시간대별 민원 발생 패턴 분석 완료! 'complaint_time_heatmap.png' 파일로 저장되었습니다.")
        
        # 가장 민원이 많은 요일과 시간대 확인
        max_day = pivot_table.sum(axis=1).idxmax()
        max_hour = pivot_table.sum(axis=0).idxmax()
        
        print(f"\n민원이 가장 많은 요일: {max_day}")
        print(f"민원이 가장 많은 시간대: {max_hour}시")
    
    # 주차장 종류별 분석 및 추천
    if '주차장종류' in parks.columns:
        type_counts = parks['주차장종류'].value_counts()
        
        print("\n주차장 종류별 분포:")
        for type_name, count in type_counts.items():
            print(f"{type_name}: {count}개 ({count/len(parks)*100:.1f}%)")
    
    # 민원 핫스팟과 주차장 부족 지역 종합 분석
    print("\n민원 발생과 주차장 관계 종합 분석:")
    
    # 결론 및 추천사항
    print("\n결론 및 추천사항:")
    print("1. 데이터 분석 결과, 주차장 부족 지역을 중심으로 새로운 주차 공간 확보가 필요합니다.")
    print("2. 민원이 많은 시간대에 맞춰 주차 관리 인력을 배치하는 것이 효율적일 수 있습니다.")
    print("3. 주차장 밀집 지역임에도 민원이 많은 곳은 주차장 정보 안내 개선이 필요할 수 있습니다.")
    print("4. 지역별 주차 수요에 맞는 맞춤형 주차 정책 수립이 필요합니다.")

comprehensive_analysis()

print("\n모든 분석이 완료되었습니다!")


====== 종합 분석 및 추천 ======
요일 및 시간대별 민원 발생 패턴 분석 완료! 'complaint_time_heatmap.png' 파일로 저장되었습니다.

민원이 가장 많은 요일: Weekday
민원이 가장 많은 시간대: 18시

주차장 종류별 분포:
NS: 757개 (51.7%)
NW: 706개 (48.3%)

민원 발생과 주차장 관계 종합 분석:

결론 및 추천사항:
1. 데이터 분석 결과, 주차장 부족 지역을 중심으로 새로운 주차 공간 확보가 필요합니다.
2. 민원이 많은 시간대에 맞춰 주차 관리 인력을 배치하는 것이 효율적일 수 있습니다.
3. 주차장 밀집 지역임에도 민원이 많은 곳은 주차장 정보 안내 개선이 필요할 수 있습니다.
4. 지역별 주차 수요에 맞는 맞춤형 주차 정책 수립이 필요합니다.

모든 분석이 완료되었습니다!
